In [30]:
import numpy as np
import numpy.ma as ma
from collections import defaultdict
import csv
import pandas as pd
from numpy import genfromtxt
import tensorflow as tf
import pickle as pickle
import tabulate
import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

In [25]:
top10Df = pd.read_csv("./data/content_top10_df.csv")
byGenreDf = pd.read_csv("./data/content_bygenre_df.csv")
byGenreDf

,genre,num movies,ave rating/genre,ratings per genre
0,Action,321,3.37,10377
1,Adventure,234,3.42,8785
2,Animation,76,3.63,2588
3,Children,69,3.44,2472
4,Comedy,326,3.36,8911
5,Crime,139,3.54,4671
6,Documentary,13,3.81,280
7,Drama,342,3.61,10201
8,Fantasy,124,3.37,4468
9,Horror,56,3.20,1345


In [26]:
def load_data():
    ''' called to load preprepared data for the lab '''
    item_train = genfromtxt('./data/content_item_train.csv', delimiter=',')
    user_train = genfromtxt('./data/content_user_train.csv', delimiter=',')
    y_train    = genfromtxt('./data/content_y_train.csv', delimiter=',')
    with open('./data/content_item_train_header.txt', newline='') as f:    #csv reader handles quoted strings better
        item_features = list(csv.reader(f))[0]
    with open('./data/content_user_train_header.txt', newline='') as f:
        user_features = list(csv.reader(f))[0]
    item_vecs = genfromtxt('./data/content_item_vecs.csv', delimiter=',')

    movie_dict = defaultdict(dict)
    count = 0
#    with open('./data/movies.csv', newline='') as csvfile:
    with open('./data/content_movie_list.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for line in reader:
            if count == 0:
                count += 1  #skip header
                #print(line) print
            else:
                count += 1
                movie_id = int(line[0])
                movie_dict[movie_id]["title"] = line[1]
                movie_dict[movie_id]["genres"] = line[2]

    with open('./data/content_user_to_genre.pickle', 'rb') as f:
        user_to_genre = pickle.load(f)

    return(item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre)

In [27]:
# Load Data, set configuration variables
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre = load_data()
num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items

In [28]:
# Scaling the trained data
item_train_unscaled = item_train
user_train_unscaled = user_train
target_train_unscaled =  y_train

# The input data are scaled with standard normalization
scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1,1))
scalerTarget.fit(y_train.reshape((-1,1)))
y_train = scalerTarget.transform(y_train.reshape((-1,1)))

print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))

True
True


In [36]:
itemTrain, itemTest = train_test_split(item_train, train_size=.8, shuffle=True, random_state=1)
userTrain, userTest = train_test_split(user_train, train_size=.8, shuffle=True, random_state=1)
yTrain, yTest        =   train_test_split(y_train, train_size=.8, shuffle=True, random_state=1)
print(f"movie/item training data shape: {itemTrain.shape}")
print(f"movie/item test data shape: {itemTest.shape}")

movie/item training data shape: (40707, 17)
movie/item test data shape: (10177, 17)


In [52]:
numOutputs = 32
tf.random.set_seed(1)
userNN = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(units=256, activation="relu"),
        tf.keras.layers.Dense(units=128, activation="relu"),
        tf.keras.layers.Dense(units=numOutputs, activation="linear")
    ]
)

itemNN = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(units=256, activation="relu"),
        tf.keras.layers.Dense(units=128, activation="relu"),
        tf.keras.layers.Dense(units=numOutputs, activation="linear"),
    ]
)

# Create the user input Neural network
inputUser = tf.keras.layers.Input(shape=(num_user_features))
vu = userNN(inputUser)
vu = tf.linalg.l2_normalize(vu, axis = 1)

# Create the item input Neural network
inputItem = tf.keras.layers.Input(shape=(num_item_features))
vm = itemNN(inputItem)
vm = tf.linalg.l2_normalize(vm, axis = 1)

# Add the dot product and output
output = tf.keras.layers.Dot(axes=1)([vu, vm])

model = tf.keras.Model([inputUser, inputItem], output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_19 (InputLayer)          [(None, 14)]         0           []                               
                                                                                                  
 input_20 (InputLayer)          [(None, 16)]         0           []                               
                                                                                                  
 sequential_16 (Sequential)     (None, 32)           40864       ['input_19[0][0]']               
                                                                                                  
 sequential_17 (Sequential)     (None, 32)           41376       ['input_20[0][0]']               
                                                                                            

In [58]:
tf.random.set_seed(1)
model.compile(
    loss = tf.keras.losses.MeanSquaredError(),
    optimizer= tf.keras.optimizers.Adam(learning_rate=.01)
)
model.fit([userTrain[:, u_s:], itemTrain[:, i_s:]], yTrain, epochs = 30)

Epoch 1/30
1273/1273 [==============================] - 3s 2ms/step - loss: 0.1238
Epoch 2/30
1273/1273 [==============================] - 2s 2ms/step - loss: 0.1150
Epoch 3/30
1273/1273 [==============================] - 2s 1ms/step - loss: 0.1108
Epoch 4/30
1273/1273 [==============================] - 2s 1ms/step - loss: 0.1063
Epoch 5/30
1273/1273 [==============================] - 2s 1ms/step - loss: 0.1038
Epoch 6/30
1273/1273 [==============================] - 2s 1ms/step - loss: 0.1007
Epoch 7/30
1273/1273 [==============================] - 2s 1ms/step - loss: 0.0985
Epoch 8/30
1273/1273 [==============================] - 2s 1ms/step - loss: 0.0962
Epoch 9/30
1273/1273 [==============================] - 2s 1ms/step - loss: 0.0940
Epoch 10/30
1273/1273 [==============================] - 2s 1ms/step - loss: 0.0924
Epoch 11/30
1273/1273 [==============================] - 2s 1ms/step - loss: 0.0906
Epoch 12/30
1273/1273 [==============================] - 2s 1ms/step - loss: 0.0888
E

In [60]:
model.evaluate([userTest[:, u_s:], itemTest[:, i_s:]], yTest)

319/319 [==============================] - 0s 1ms/step - loss: 0.0817


0.08165575563907623

In [71]:
def gen_user_vecs(user_vec, num_items):
    """ given a user vector return:
        user predict maxtrix to match the size of item_vecs """
    user_vecs = np.tile(user_vec, (num_items, 1))
    return user_vecs
def get_user_vecs(user_id, user_train, item_vecs, user_to_genre):
    """ given a user_id, return:
        user train/predict matrix to match the size of item_vecs
        y vector with ratings for all rated movies and 0 for others of size item_vecs """

    if not user_id in user_to_genre:
        print("error: unknown user id")
        return None
    else:
        user_vec_found = False
        for i in range(len(user_train)):
            if user_train[i, 0] == user_id:
                user_vec = user_train[i]
                user_vec_found = True
                break
        if not user_vec_found:
            print("error in get_user_vecs, did not find uid in user_train")
        num_items = len(item_vecs)
        user_vecs = np.tile(user_vec, (num_items, 1))

        y = np.zeros(num_items)
        for i in range(num_items):  # walk through movies in item_vecs and get the movies, see if user has rated them
            movie_id = item_vecs[i, 0]
            if movie_id in user_to_genre[user_id]['movies']:
                rating = user_to_genre[user_id]['movies'][movie_id]
            else:
                rating = 0
            y[i] = rating
    return(user_vecs, y)
# Prediction for a new user
new_user_id = 5000
new_rating_ave = 0.0
new_action = 0.0
new_adventure = 5.0
new_animation = 0.0
new_childrens = 0.0
new_comedy = 0.0
new_crime = 0.0
new_documentary = 0.0
new_drama = 0.0
new_fantasy = 5.0
new_horror = 0.0
new_mystery = 0.0
new_romance = 0.0
new_scifi = 0.0
new_thriller = 0.0
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])

# generate and replicate the user vector to match the number movies in the data set.
user_vecs = gen_user_vecs(user_vec,len(item_vecs))
# generate and replicate the user vector to match the number movies in the data set.
user_vecs = gen_user_vecs(user_vec,len(item_vecs))

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction 
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #using unscaled vectors for display

uid = 2 
# form a set of user vectors. This is the same vector, transformed and repeated.
user_vecs, y_vecs = get_user_vecs(uid, user_train_unscaled, item_vecs, user_to_genre)

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction 
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #using unscaled vectors for display
sorted_user  = user_vecs[sorted_index]
sorted_y     = y_vecs[sorted_index]




27/27 [==============================] - 0s 2ms/step
